# This will be the main file for training our model for general anxiety presence detection in patients. 



Credits to Data Flair for the inspiration for the code.
Their guide on training speech emotion recognition models can be found here: https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/

In [1]:
import scipy
import numpy as np
import matplotlib as plt
import librosa
import pathlib
import soundfile
import os, glob, pickle
from os import path
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
emotions = {
    "anx": "anxious",
    "not": "not anxious"
}

In [89]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        
        if chroma:
            stft=np.abs(librosa.stft(X))
            
        result=np.array([])
        
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
            
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
            
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
            
    return result

In [91]:
def load_data(test_size = 0.2):
    x, y = [], []
    
    for file in glob.glob("C:\\Users\\School\\Documents\\Environments\\v\\Data\\*_*_AUDIO.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("_")[0]]
        feature=extract_feature(file, mfcc=False, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
        
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [92]:
x_train,x_test,y_train,y_test=load_data(test_size=0.2)
print((x_train.shape[0], x_test.shape[0]))
print(f'Features extracted: {x_train.shape[1]}')

(84, 22)
Features extracted: 140


In [95]:
model=MLPClassifier(alpha=0.01, batch_size='auto', epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=2000)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 72.73%


In [96]:
model_file_name = "trained model.sav"
pickle.dump(model, open(model_file_name, 'wb'))